In [2]:
# https://www.kaggle.com/shixw125/grupo-bimbo-inventory-demand/log-mean-plus-lb-0-47/code

library(data.table)

print("Reading data")
train <- fread('./input-data/train.csv', 
               select = c('Cliente_ID', 'Producto_ID', 'Agencia_ID', 'Ruta_SAK', 'Demanda_uni_equil'))

test <- fread('./input-data/test.csv', 
              select = c('id', 'Cliente_ID', 'Producto_ID', 'Agencia_ID', 'Ruta_SAK'))


[1] "Reading data"
Read 74180464 rows and 5 (of 11) columns from 2.980 GB file in 00:00:25
Read 6999251 rows and 5 (of 7) columns from 0.234 GB file in 00:00:03


In [3]:
print("Computing means")
#transform target variable to log(1 + demand) - this makes sense since we're 
#trying to minimize rmsle and the mean minimizes rmse:
train$log_demand = 1.005*log1p(train$Demanda_uni_equil)-0.005
mean_total <- mean(train$log_demand) #overall mean

#mean by product
mean_P <-  train[, .(MP = mean(log_demand)), by = .(Producto_ID)]
#mean by ruta
mean_C <-  train[, .(MC = mean(log_demand)), by = .(Cliente_ID)]
#mean by product and agencia
mean_PA <- train[, .(MPA = mean(log_demand)), by = .(Producto_ID, Agencia_ID)] 
#mean by product and ruta
mean_PR <- train[, .(MPR = mean(log_demand)), by = .(Producto_ID, Ruta_SAK)] 
#mean by product, client, agencia
mean_PCA <- train[, .(MPCA = mean(log_demand)), by = .(Producto_ID, Cliente_ID, Agencia_ID)]


[1] "Computing means"


In [4]:
print("Merging means with test set")
submit <- merge(test, mean_PCA, all.x = TRUE, by = c("Producto_ID", "Cliente_ID", "Agencia_ID"))
submit <- merge(submit, mean_PR, all.x = TRUE, by = c("Producto_ID", "Ruta_SAK"))
submit <- merge(submit, mean_PA, all.x = TRUE, by = c("Producto_ID", "Agencia_ID"))
submit <- merge(submit, mean_C, all.x = TRUE, by = "Cliente_ID")
submit <- merge(submit, mean_P, all.x = TRUE, by = "Producto_ID")

[1] "Merging means with test set"


In [5]:
# Now create Predictions column;
submit$Pred <- expm1(submit$MPCA)*0.72+expm1(submit$MPR)*0.19+0.1
submit[is.na(Pred)]$Pred <- expm1(submit[is.na(Pred)]$MPR)*0.746+0.17
submit[is.na(Pred)]$Pred <- expm1(submit[is.na(Pred)]$MC)*0.82+0.86
submit[is.na(Pred)]$Pred <- expm1(submit[is.na(Pred)]$MPA)*0.54+0.8
submit[is.na(Pred)]$Pred <- expm1(submit[is.na(Pred)]$MP)*0.44+1
submit[is.na(Pred)]$Pred <- expm1(mean_total)

# this is the code to calculate coefficients

submit$Pred <- round(submit$Pred,5)

In [6]:
print("Write out submission file")
# now relabel columns ready for creatig submission
setnames(submit,"Pred","Demanda_uni_equil")
# Any results you write to the current directory are saved as output.
write.csv(submit[,.(id,Demanda_uni_equil)],"./Submissions/submission-Log_Mean_Plus.csv", row.names = FALSE)
print("Done!")


[1] "Write out submission file"
[1] "Done!"
